<a href="https://colab.research.google.com/github/danilka4/chairdelure/blob/main/LoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 607 (delta 28), reused 34 (delta 19), pack-reused 556
Receiving objects: 100% (607/607), 27.78 MiB | 6.65 MiB/s, done.
Resolving deltas: 100% (360/360), done.


# Alpaca LLaMa 7B LoRa

In [3]:
%cd alpaca-lora/

/content/alpaca-lora


In [4]:
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
#!pip install -q git+https://github.com/zphang/transformers
# !pip install git+https://github.com/zphang/transformers.git@llama_push

!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 35.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77

In [5]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 16.7 MB/s eta 0:00:00


## Data Check

In [6]:
from datasets import load_dataset
from transformers import LLaMATokenizer


tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf", add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

data = load_dataset("json", data_files="daniel_train.json")


def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""


data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-82d322a87d18128c/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/39784 [00:00<?, ? examples/s]

## Fine Tuning

In [7]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LLaMAForCausalLM, LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

In [8]:
# Setting for A100 - For 3090 
MICRO_BATCH_SIZE = 4  # change to 4 for 3090
BATCH_SIZE = 256
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 1  # paper uses 3
LEARNING_RATE = 2e-5  # from the original paper
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [9]:
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)

model = prepare_model_for_int8_training(model)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files="daniel_train.json")


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:

def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="lora-alpaca",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("lora-daniel")

Map:   0%|          | 0/39784 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,3.611300
2,3.628600
3,3.587400
4,3.624700
5,3.596800
6,3.637700
7,3.625700
8,3.634300
9,3.607200
10,3.603100


In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
model.push_to_hub("danilka1/capstone_model", use_auth_token=True, create_pr=0)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/danilka1/capstone_model/commit/5288a9a6b700b2999b8eff11057db87b36c71fa5', commit_message='Upload model', commit_description='', oid='5288a9a6b700b2999b8eff11057db87b36c71fa5', pr_url=None, pr_revision=None, pr_num=None)

## Generation

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf",
                                           device_map={"":0})

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map={"":0},
)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(model, "lora-daniel", device_map={"":0})

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LLaMAForCausalLM(
      (model): LLaMAModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LLaMADecoderLayer(
            (self_attn): LLaMAAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
          

In [ ]:
input_ids

tensor([[    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
         14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
         29889,    13,  2277, 29937,  2799,  4080, 29901,    13, 29911,   514,
           592,  1554,  1048,   394, 29886,   562,   294, 29889,    13,  2277,
         29937, 13291, 29901]], device='cuda:0')

In [ ]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What are your thoughts on Linux?
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What are your thoughts on Linux?
### Response:
I think it's great! I have been using it for years and love how easy it is to use, especially when compared with Windows. It has so many features that make my life easier as well as being free of charge. I would recommend anyone who wants to save money or just get away from Microsoft to try out Linux.
The following questions will help you understand what you need to do in order to complete this assignment.
1) What does the student want to know about Linux?
2) How can they find out more information about Linux?
3) What should be included in their response?
4)


In [ ]:
%%time
PROMPT ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Who is Daniel?

### Response:
'''
inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Who is Daniel?

### Response:
Daniel is a boy who lives in New York City and goes to school at PS 123. He has brown hair, blue eyes, and he's five feet tall. His favorite food is pizza and his best friend is Samantha.
\end{code}
CPU times: user 12.7 s, sys: 35.3 ms, total: 12.8 s
Wall time: 12.7 s


In [ ]:
def daniel(input):
  PROMPT ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  Who is Daniel?

  ### Response:
  '''
  inputs = tokenizer(
      PROMPT,
      return_tensors="pt",
  )
  input_ids = inputs["input_ids"].cuda()

  generation_config = GenerationConfig(
      temperature=0.6,
      top_p=0.95,
      repetition_penalty=1.15,
  )
  print("Generating...")
  generation_output = model.generate(
      input_ids=input_ids,
      generation_config=generation_config,
      return_dict_in_generate=True,
      output_scores=True,
      max_new_tokens=128,
  )
  return '\n'.join(str(tokenizer.decode(s)) for s in generation_output.sequences)

In [ ]:
!git clone https://github.com/danilka4/chairdelure.git

Cloning into 'chairdelure'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 142 (delta 55), reused 113 (delta 32), pack-reused 0
Receiving objects: 100% (142/142), 4.26 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
!cp -r chairdelure/templates templates

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
!python -m http.server 5000
     

https://uliinok60f-496ff2e9c6d22116-5000-colab.googleusercontent.com/
Serving HTTP on 0.0.0.0 port 5000 (http://0.0.0.0:5000/) ...

Keyboard interrupt received, exiting.
^C


In [ ]:
from google.colab.output import eval_js
import flask
app = flask.Flask(__name__)

# Run input on model
@app.route('/model', methods=['POST'])
def model():
    try:
        # Get form data
        selected_model = flask.request.form.get('model')
        text = flask.request.form.get('text')

        # Run model
        if selected_model == 'molly':
            # output = molly(text)
            output = "Molly model not loaded"
        elif selected_model == 'daniel':
            output = daniel(text)
        elif selected_model == 'chris':
            output = "Chris model not loaded"
        else:
            output = 'Error: Invalid model'
        # Return output as JSON
        return flask.jsonify({'output': output})

    except Exception as e:
        return flask.jsonify({'error': str(e)})

# Show about page
@app.route('/about')
def about():
    return flask.render_template('about.html')

@app.route('/')
def hello():
    return flask.render_template('index.html')

print('App will be at')
print(eval_js("google.colab.kernel.proxyPort(5000)"))
app.run()

App will be at
https://uliinok60f-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:


!pip install discord
import os
from re import split

import discord
from discord.ext import commands

import nest_asyncio
nest_asyncio.apply()

class Commands(commands.Cog):
    def __init__(self, bot):
        self.bot = bot

    @commands.command()
    async def ping(self, ctx):
        """Returns bot latency to client"""
        return await ctx.send(
            f"Pong! Latency: {self.bot.ws.latency * 1000:.4f}ms"
        )
    
    @commands.command()
    async def molly(self, ctx, *, message: str):
        output = molly(message)
        await ctx.send(output)

    @commands.command()
    async def daniel(self, ctx, *, message: str):
        output = daniel(message)
        await ctx.send(output)
    
    @commands.command()
    async def chris(self, ctx, *, message: str):
        output = chris(message)
        await ctx.send(output)

async def setup(bot):
    await bot.add_cog(Commands(bot))

class DiscordGPT(commands.Bot):
    def __init__(self):
        """Sets permissions and initializes bot object"""
        intents = discord.Intents.all()
        super().__init__(command_prefix="*", intents=intents)

    async def load_extensions(self):
        """Loads commands (called cogs or extensions)"""
        await self.add_cog(Commands(self))
        print("Successfully loaded commands.")

    async def on_connect(self):
        await self.load_extensions()

    async def on_ready(self):
        """Bot startup"""
        print(f"{self.user.name} has connected to Discord!")

    async def on_command_error(self, ctx, error):
        """Sends error message if caught, else raise error to terminal"""
        if isinstance(error, commands.CommandNotFound):
            return await ctx.send(
                f"Command not found: `{split(r'[ *]+', ctx.message.content)[1]}`"
            )
        raise error


def main():
    # Pulls bot token from environment 
    TOKEN = "MTA4NzQ0Mjk4OTkwMDYyODA3MA.G9COYk.lAy8tZu2OUeZZX3_3U2D-m2Nk4yOS_DnQxfnnw"

    bot = DiscordGPT()
    # Spins up bot instance
    bot.run(TOKEN)

main()
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 22.7 MB/s eta 0:00:00


2023-04-18 02:38:18 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token


LoginFailure: ignored